In [2]:
#rm -rf out/nll/backdoor-10000-2/

In [3]:
import glob
import itertools
import sys
sys.path.append('..')

import torch as T

from src import metrics
from src import utils
from src.ds import CausalGraph
from src.scm import (SCM, CTM, DiscreteDistribution,
                     PartiallyCorrelatedDistribution, Distribution)
from src.nn import Simple, NCM, MMDModule, NLLModule, UniformDistribution

cg_files = sorted(glob.glob('../dat/cg/*'))
cg_files = [
    '../dat/cg/backdoor.cg',
    '../dat/cg/bow.cg',
    '../dat/cg/frontdoor.cg',
    '../dat/cg/iv.cg',
    '../dat/cg/m.cg',
    '../dat/cg/napkin.cg',
    '../dat/cg/simple.cg']
ns = [1000, 10000, 100000]

for i, (n, cg_file) in enumerate(itertools.product(ns, cg_files)):
    while True:
        try:
            if not utils.pipeline(cg_file, n, n_trials=20, name='nll', module=NLLModule):
                break
        except AssertionError as e:
            print(e)
            print('Failed', n, cg_file)

out/nll/backdoor-1000-0
0.20264998078346252 0.2022348940372467 0.0004150867462158203
0.20200175046920776 0.2007806897163391 0.0012210607528686523
0.20720919966697693 0.20464149117469788 0.0025677084922790527
0.21306461095809937 0.20834702253341675 0.004717588424682617
0.21891069412231445 0.21087971329689026 0.008030980825424194
0.22459855675697327 0.2115994095802307 0.012999147176742554
0.23081257939338684 0.2105211317539215 0.020291447639465332
0.24130475521087646 0.21042263507843018 0.03088212013244629
0.25799232721328735 0.21177849173545837 0.04621383547782898
0.27733349800109863 0.20926782488822937 0.06806567311286926
0.2970873713493347 0.198600172996521 0.09848719835281372
0.3160453140735626 0.176437646150589 0.13960766792297363
0.33338016271591187 0.14021089673042297 0.1931692659854889
0.3483545780181885 0.08859282732009888 0.2597617506980896
ctm table


,Z,X,Y,P(V)
0,0,0,0,0.079368
1,0,0,1,0.008912
2,0,1,0,0.420264
3,0,1,1,0.408432
4,1,0,0,0.009344
5,1,0,1,0.070397
6,1,1,0,0.000347
7,1,1,1,0.002935


observed table


,Z,X,Y,P(V)
0,0,0,0,0.085
1,0,0,1,0.010
2,0,1,0,0.430
3,0,1,1,0.398
4,1,0,0,0.006
5,1,0,1,0.070
6,1,1,1,0.001


moved out/nll/backdoor-1000-0 to err/nll/backdoor-1000-3


AttributeError: module 'src.metrics' has no attribute 'obs_effect'

In [5]:
T.nn.ParameterDict()

torch.nn.modules.container.ParameterDict

In [4]:
import torch as T

In [ ]:
ctm

In [8]:
d = T.load('out/mmd/dat-10000-0/best.th')
g = 'backdoor'
m = MMDModule(CTM(utils.RPA[g]), NCM(CausalGraph.read(f'../dat/cg/{g}.cg')))
m.load_state_dict(d)

<All keys matched successfully>

In [71]:
df = pd.DataFrame(results, index=pd.Index(list(map(lambda s: s.split('/')[-1].split('.')[0], cg_files)), name='graph'))
df['ate_error'] = df.true_ate - df.ncm_ate
df['obs_effect_error'] = df.true_obs_effect - df.ncm_obs_effect
df

,true_ate,ncm_ate,true_obs_effect,ncm_obs_effect,kl,supremum_norm,ate_error,obs_effect_error
graph,,,,,,,,
backdoor,0.220649,0.216957,0.290053,0.285144,0.002112,0.012266,0.003692,0.004909
bow,0.240343,0.431371,0.295077,0.297633,0.000548,0.010598,-0.191028,-0.002556
frontdoor,0.076267,0.103640,0.129549,0.117969,0.004057,0.018960,-0.027373,0.011579
iv,0.241844,0.302528,0.280848,0.272303,0.001589,0.013361,-0.060684,0.008545
m,0.240631,0.260124,0.241369,0.260296,0.005861,0.022149,-0.019493,-0.018926
napkin,0.239342,0.240120,0.245026,0.245253,0.002711,0.008153,-0.000778,-0.000227
simple,0.315517,0.336063,0.316806,0.335472,0.000392,0.010174,-0.020546,-0.018666


Next steps:

- Run 5-10 trials each.
- Use multiple epochs / less data.
- Sample CTMs as data-generating model. Talk to Adèle.
- Add # samples; try multiple sample sizes.
- Plot posterior of $P(Y | do(X))$. Hypothesis: negative example posteriors will be diluted / high-variance.
- Sample a graphical structure? (later)
- Plot results.

Next experimental set: compare with WERM and Plug-in.

Delta:
- We can have bidirected + directed arrows between a pair of variables.